<div align="center">
<p align="center" style="width: 100%;">
    <img src="https://raw.githubusercontent.com/vlm-run/.github/refs/heads/main/profile/assets/vlm-black.svg" alt="VLM Run Logo" width="80" style="margin-bottom: -5px; color: #2e3138; vertical-align: middle; padding-right: 5px;"><br>
</p>
<p align="center"><a href="https://docs.vlm.run"><b>Website</b></a> | <a href="https://docs.vlm.run/"><b>API Docs</b></a> | <a href="https://docs.vlm.run/blog"><b>Blog</b></a> | <a href="https://discord.gg/AMApC2UzVY"><b>Discord</b></a> | <a href="https://chat.vlm.run"><b>Chat</b></a>
</p>
</div>

# VLM Run Orion - Botanical Pathology Analysis

This notebook demonstrates how to use [VLM Run Orion's](https://vlm.run/orion) vision capabilities to act as a Plant Pathologist. You can provide a photo of a distressed plant leaf and ask the model to identify the pathogen and suggest a treatment plan.

For more details on the API, see the [Agent API docs](https://docs.vlm.run/agents/introduction).

## Prerequisites

- Python 3.10+
- VLM Run API key (get one at [app.vlm.run](https://app.vlm.run))
- VLM Run Python Client with OpenAI extra `vlmrun[openai]`

## Setup

First, install the required packages and configure the environment.

In [1]:
# Install required packages
%pip install vlmrun[openai] --upgrade --quiet
%pip install cachetools pillow requests numpy --quiet

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import os
import getpass

VLMRUN_API_KEY = os.getenv("VLMRUN_API_KEY", None)
if VLMRUN_API_KEY is None:
    VLMRUN_API_KEY = getpass.getpass("Enter your VLM Run API key: ")

## Initialize the VLM Run Client

We use the OpenAI-compatible chat completions interface through the VLM Run SDK.

In [3]:
from vlmrun.client import VLMRun

BASE_URL = os.getenv("VLMRUN_BASE_URL", "https://agent.vlm.run/v1")
client = VLMRun(api_key=VLMRUN_API_KEY, base_url=BASE_URL)
print("VLM Run client initialized successfully!")
print(f"Base URL: {BASE_URL}")

VLM Run client initialized successfully!
Base URL: https://agent.vlm.run/v1


## Response Models

We define Pydantic models for structured outputs. The report will be returned as a structured object containing specific sections like Symptom Analysis, Pathogen ID, and Treatment Plans.

In [4]:
from typing import List
from pydantic import BaseModel, Field


class SymptomAnalysis(BaseModel):
    """Visual analysis of the distress symptoms."""
    lesion_morphology: str = Field(..., description="Shape, size, and color of spots (e.g., necrotic centers, yellow halos)")
    patterning: str = Field(..., description="Whether discoloration follows veins or spreads across them")
    fruiting_bodies_signs: str = Field(..., description="Visible mold, spores, or bacterial ooze")


class TreatmentPlan(BaseModel):
    """Comprehensive management strategy."""
    cultural_controls: List[str] = Field(..., description="Immediate physical actions like pruning or irrigation changes")
    chemical_biological_treatments: List[str] = Field(..., description="Specific active ingredients and application instructions")
    prevention: List[str] = Field(..., description="Recommendations to prevent recurrence")


class PlantPathologyReport(BaseModel):
    """Structured report for plant disease identification and management."""
    symptom_analysis: SymptomAnalysis = Field(..., description="Detailed visual analysis of symptoms")
    pathogen_identification: str = Field(..., description="Most likely scientific name of the fungal or bacterial pathogen")
    disease_progression: str = Field(..., description="Explanation of how the disease affects the plant's physiology")
    treatment_plan: TreatmentPlan = Field(..., description="Step-by-step management and treatment plan")


print("Response models defined successfully!")

Response models defined successfully!


## Helper Functions

We create helper functions to simplify making chat completion requests with structured outputs.

In [5]:
import hashlib
import json
from typing import Any, Type, TypeVar

import cachetools
from vlmrun.common.image import encode_image
from PIL import Image


T = TypeVar('T', bound=BaseModel)


def custom_key(prompt: str, image_path: str | None = None, response_model: Type[T] | None = None, model: str = "vlmrun-orion-1:auto"):
    """Custom key for caching chat_completion."""
    response_key = hashlib.sha256(json.dumps(response_model.model_json_schema(), sort_keys=True).encode()).hexdigest() if response_model else ""
    image_key = hashlib.sha256(image_path.encode()).hexdigest() if image_path else ""
    return (prompt, image_key, response_key, model)


@cachetools.cached(cache=cachetools.TTLCache(maxsize=100, ttl=3600), key=custom_key)
def chat_completion(
    prompt: str,
    image_path: str | None = None,
    response_model: Type[T] | None = None,
    model: str = "vlmrun-orion-1:auto"
) -> tuple[BaseModel | str, str]:
    """
    Make a chat completion request with structured output for Plant Pathology.

    Args:
        prompt: The prompt describing the analysis task
        image_path: Path to the image file (Plant leaf)
        response_model: Pydantic model for structured output
        model: Model to use (default: vlmrun-orion-1:auto)

    Returns:
        Tuple of (parsed response model or text, session_id)
    """
    content = [{"type": "text", "text": prompt}]
    
    # Add image if provided
    if image_path:
        image = Image.open(image_path)
        image_data = encode_image(image, format="JPEG")
        content.append({"type": "image_url", "image_url": {"url": image_data}})

    kwargs = {
        "model": model,
        "messages": [{"role": "user", "content": content}]
    }

    if response_model:
        kwargs["response_format"] = {
            "type": "json_schema",
            "schema": response_model.model_json_schema()
        }

    response = client.agent.completions.create(**kwargs)
    response_text = response.choices[0].message.content
    session_id = response.session_id

    if response_model:
        result = response_model.model_validate_json(response_text)
        return result, session_id

    return response_text, session_id

print("Helper functions defined!")

Helper functions defined!


## Botanical Pathology Analysis

Provide a photo of a plant leaf with spots or discoloration to identify the pathogen and get a treatment plan.

In [6]:
# Prompt for Plant Pathology
PATHOLOGY_PROMPT = """
Task: Act as a Plant Pathologist. Analyze the attached image of a distressed plant leaf to identify the likely pathogen and provide a comprehensive management strategy.

Instructions:

Symptom Analysis: Describe the visual characteristics of the distress. Specifically, look for:

Lesion Morphology: Shape (e.g., circular, angular), size, and color of spots (e.g., necrotic centers, yellow halos).

Patterning: Note if the discoloration follows leaf veins (often bacterial) or spreads across them (often fungal).

Signs of Fruiting Bodies: Identify any visible mold, spores, or bacterial ooze.

Pathogen Identification: Based on the visual evidence and host plant type, provide the most likely scientific name of the fungal or bacterial pathogen (e.g., Cercospora, Pseudomonas syringae, Phytophthora).

Disease Progression: Briefly explain how this specific disease affects the plant's physiology (e.g., "inhibits photosynthesis," "disrupts vascular transport").

Treatment and Management Plan: * Cultural Controls: Immediate physical actions (e.g., "pruning infected foliage," "improving air circulation," "adjusting irrigation").

Chemical/Biological Treatments: Suggest specific active ingredients (e.g., copper-based fungicides, Neem oil, or specific antibiotics) and application instructions.

Prevention: Recommendations to prevent recurrence in future growing seasons.
"""

print("Pathology prompt prepared!")
print(f"\\nPrompt length: {len(PATHOLOGY_PROMPT)} characters")

Pathology prompt prepared!
\nPrompt length: 1377 characters


In [ ]:
image_path = "leaf.jpeg" 

if os.path.exists(image_path):
    result, session_id = chat_completion(
        prompt=PATHOLOGY_PROMPT,
        image_path=image_path,
        response_model=PlantPathologyReport,
        model="vlmrun-orion-1:auto"
    )

    print(">> RESPONSE")
    print(result)
    print(f"\\n>> SESSION ID: {session_id}")
    
    print("\\n>> BOTANICAL PATHOLOGY REPORT")
    print("=" * 80)
    print(f"Pathogen: {result.pathogen_identification}\\n")
    print("Symptom Analysis:")
    print(f"- Morphology: {result.symptom_analysis.lesion_morphology}")
    print(f"- Patterning: {result.symptom_analysis.patterning}")
    print(f"- Signs: {result.symptom_analysis.fruiting_bodies_signs}\\n")
    print(f"Disease Progression: {result.disease_progression}\\n")
    print("Treatment Plan:")
    print("Cultural:")
    for ctrl in result.treatment_plan.cultural_controls:
        print(f"- {ctrl}")
    print("\\nChemical/Biological:")
    for chem in result.treatment_plan.chemical_biological_treatments:
        print(f"- {chem}")
    print("\\nPrevention:")
    for prev in result.treatment_plan.prevention:
        print(f"- {prev}")
    print("=" * 80)
else:
    print(f"Image not found at {image_path}. Please add a plant leaf image to run this example.")

>> RESPONSE
symptom_analysis=SymptomAnalysis(lesion_morphology='Irregular to rounded dark brown/black necrotic centers (2-5mm) surrounded by prominent, diffuse yellow chlorotic halos.', patterning="The lesions are scattered and cross-veinal, meaning they are not restricted by the leaf's primary or secondary veins.", fruiting_bodies_signs='No macroscopic fruiting bodies, mold, or bacterial ooze are currently visible; however, microscopic analysis would likely reveal conidiophores in the necrotic centers.') pathogen_identification='Cercospora hibisci' disease_progression="Cercospora hibisci is a fungal pathogen that colonizes the leaf tissue, often utilizing the light-activated toxin cercosporin to induce cell death. This necrotic tissue reduces the leaf's effective surface area for photosynthesis. As the disease progresses, it triggers premature senescence and abscission (leaf drop), which can significantly weaken the host plant, reduce its aesthetic value, and in severe cases, limit it

---

## Conclusion

This notebook demonstrated how to use **VLM Run Orion** to identify plant diseases and generate treatment plans.

### Key Takeaways

1. **Expert Systems**: By prompting the model to act as a specific expert (Plant Pathologist), we ensure the output is technically accurate and well-structured.
2. **Visual Diagnosis**: The model combines visual features (lesions, patterns) to deduce the underlying pathogen.
3. **Actionable Advice**: The output is not just a label but a complete management guide for the grower.

### Next Steps

- Upload different plant species to test the model's adaptability.
- Integrate with a real-time camera feed for a field diagnostic app.
- Explore the [VLM Run Documentation](https://docs.vlm.run) for more capabilities.

Happy gardening! 🌿